In [1]:
from t9_data_preparer import T9DataPreparer

In [2]:
preparer = T9DataPreparer()

In [3]:
samples = preparer.extract_samples_from_files()
print(f"Extracted {len(samples)} samples")
print(f"Samples example: {samples[0]}")

Extracted 43674 samples
Samples example: ('7858', 'qulu', '去路')


In [4]:
sentences = preparer.extract_sentences_from_files()
print(f"Extracted {len(sentences)} sentences")
print(f"Sentences example: {sentences[0]}")

Extracted 45703 sentences
Sentences example: 云在天空游荡，它从远方飘来，又飘向远方。


In [5]:
output_path = 'data/zh_T9_dataset.tsv'
with open(output_path, 'w', encoding='utf-8') as f:
    for digit_seq, pinyin_seq, hanzi in samples:
        f.write(f"{digit_seq}\t{pinyin_seq}\t{hanzi}\n")

In [6]:
with open('data/zh_sent_dataset.tsv', 'w', encoding='utf-8') as f:
    for sent in sentences:
        f.write(sent + '\n')